# BM25 Retrieval with SDM

### Step 1: Import everything and load variables

In [1]:
#pip install python-terrier

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib64/jvm/java"
os.environ["PYTERRIER_VERSION"] = "snapshot"
os.environ["PYTERRIER_HELPER_VERSION"] = "snapshot"
import pyterrier as pt
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
#import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


I will use a small hardcoded example located in ./iranthology-dataset-tira.
The output directory is /tmp/


### Step 2: Load the Data

In [4]:
print('Step 2: Load the data.')

queries = pt.io.read_topics('milestone-1-topics.xml', format='trecxml')

documents = [json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r')]
documents = [{'docno': i['docno'], 'text': i['text'], 'title': i['original_document']['title'], 'abstract': i['original_document']['abstract']} for i in documents]

print('We look at the first document:\n')
print(documents[0])

Step 2: Load the data.
We look at the first document:

{'docno': '2019.sigirconf_workshop-2019birndl.0', 'text': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019 ', 'title': 'Proceedings of the 4th Joint Workshop on Bibliometric-enhanced Information Retrieval and Natural Language Processing for Digital Libraries (BIRNDL 2019) co-located with the 42nd International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR 2019), Paris, France, July 25, 2019', 'abstract': ''}


### Step 3: Create the Index

In [5]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100, 'title': 10240, 'abstract': 10240, 'text': 10240}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

Step 3: Create the Index.
rm: das Entfernen von './index/.nfs000000000022133e0000003e' ist nicht möglich: Das Gerät oder die Ressource ist belegt
rm: das Entfernen von './index/.nfs000000000022123300000040' ist nicht möglich: Das Gerät oder die Ressource ist belegt
rm: das Entfernen von './index/.nfs00000000002211c400000041' ist nicht möglich: Das Gerät oder die Ressource ist belegt
rm: das Entfernen von './index/.nfs0000000000220efe0000003f' ist nicht möglich: Das Gerät oder die Ressource ist belegt


 31%|███████████████████▋                                           | 16722/53673 [00:05<00:06, 5373.16it/s]

14:50:28.522 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


100%|███████████████████████████████████████████████████████████████| 53673/53673 [00:12<00:00, 4137.23it/s]


14:50:38.240 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


### Step 4: Create Retrieval Pipeline

In [17]:
sdm = pt.rewrite.SDM()

dph = pt.BatchRetrieve(index_ref, wmodel="DPH", verbose=True, metadata=['docno', 'text', 'title', 'abstract'])

bm25_title = pt.text.scorer(body_attr="title", wmodel="BM25")
bm25_abstract = pt.text.scorer(body_attr="abstract", wmodel="BM25")
bm25_text = pt.text.scorer(body_attr="text", wmodel="BM25")

combined_bm25_score = ((2*bm25_title) + (0.5*bm25_abstract) + (1*bm25_text))


dph_title = pt.text.scorer(body_attr="title", wmodel="DPH")
dph_abstract = pt.text.scorer(body_attr="abstract", wmodel="DPH")
dph_text = pt.text.scorer(body_attr="text", wmodel="DPH")


combined_dph_score = ((2*dph_title) + (0.5*dph_abstract) + (1*dph_text))


retrieval_pipeline = sdm >> dph%100 >> combined_bm25_score + combined_dph_score

In [19]:
print('We show the first ten query document pairs after BM25 retrieval to show what fields we have added:')
run = dph(queries)
run.head(5)

We show the first ten query document pairs after BM25 retrieval to show what fields we have added:


BR(DPH): 100%|█████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.55q/s]


qid  docid                                              docno   
0   1  33491                  2018.wwwconf_conference-2018c.140  \
1   1   7566                 2008.sigirconf_conference-2008.116   
2   1  48967  2011.sigirjournals_journal-ir0anthology0volume...   
3   1  14643                      2019.wsdm_conference-2019.115   
4   1  24299                        2017.cikm_conference-2017.2   

                                                text   
0  Fake News Detection in Social Networks via Cro...  \
1  Detecting synonyms in social tagging systems t...   
2  Improving search effectiveness in sentence ret...   
3  Fake News: Fundamental Theories, Detection Str...   
4  Deception Detection: When Computers Become Bet...   

                                               title   
0  Fake News Detection in Social Networks via Cro...  \
1  Detecting synonyms in social tagging systems t...   
2  Improving search effectiveness in sentence ret...   
3  Fake News: Fundamental Theories, Detection Str...   
4  Deception Detection: When Computers Become Bet...   

                                            abstract  rank      score   
0  ABSTRACTOur work considers leveraging crowd si...     0  16.149411  \
1  ABSTRACTCollaborative tagging used in online s...     1  15.035063   
2  AbstractIn this thesis we study thoroughly sen...     2  14.634735   
3  ABSTRACTThe explosive growth of fake news and ...     3  14.609710   
4  ABSTRACTWhether we like it or not, deception h...     4  14.343040   

                                               query  
0   relevant documents show the bigram of fake ne...  
1   relevant documents show the bigram of fake ne...  
2   relevant documents show the bigram of fake ne...  
3   relevant documents show the bigram of fake ne...  
4   relevant documents show the bigram of fake ne...

### Step 5: Create the run

In [ ]:
print('Step 5: Create Run.')

run = retrieval_pipeline(queries)

In [ ]:
print('We look at the first 10 results of the run (query has ben expanded):\n')
run.head(10)

### Step 6: Persist Run

In [ ]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='multi-field', depth=1000)

print('Done :)')